In [1]:
import pickle
import pandas as pd

import win32com.client

One series data인 KODEX200 price 부터 구해본다. 

추후 scale과 reusability를 고려하여 작성

## df에서 코드 가져오기

In [2]:
kospi_ETF_df = pd.read_pickle('./data_files/kospi_ETF_df.pkl')
kospi_ETF_df.head()

,code,name,stock_control_kind,stock_supervision_kind,stock_status_kind,stock_kospi200_kind,stock_section_kind
686,A069500,KODEX 200,0,0,0,0,10
689,A069660,KOSEF 200,0,0,0,0,10
747,A091160,KODEX 반도체,0,0,0,0,10
748,A091170,KODEX 은행,0,0,0,0,10
749,A091180,KODEX 자동차,0,0,0,0,10


In [3]:
kospi_ETF_df.name

686                     KODEX 200
689                     KOSEF 200
747                     KODEX 반도체
748                      KODEX 은행
749                     KODEX 자동차
                  ...            
1361      KBSTAR KRX국채선물3년10년스티프너
1362    KBSTAR KRX국채선물3년10년스티프너2X
1363      KBSTAR KRX국채선물3년10년플래트너
1364    KBSTAR KRX국채선물3년10년플래트너2X
1366             HANARO KAP초장기국고채
Name: name, Length: 451, dtype: object

In [4]:
code = kospi_ETF_df[kospi_ETF_df.name == 'KODEX 200'].to_numpy()[0][0]
code

'A069500'

## 종목과 조회조건 설정 (Daily data)

In [5]:
instStockChart = win32com.client.Dispatch('CPSysDib.StockChart')

In [23]:
# 종목과 조회조건 등을 설정. 

instStockChart.SetInputValue(0, code) # 첫 인자는 데이터 타입, 두 번째 인자는 입력 데이터 값. 0은 종목코드, A003540은 종목코드. 
instStockChart.SetInputValue(1, ord('2')) # 첫 번째 인자는 기간/요청 설정을 말하는 1. 두 번째 인자는 기간으로 요청 시 1, 개수로 요청시 2. 
instStockChart.SetInputValue(4, 1000) # 첫 번째 인자는 요청개수 설정을 말하는 4. 두 번째 인자는 실제로 몇 개 요청할 것인지 개수. (여기선 10개)
instStockChart.SetInputValue(5, [0, 2, 3, 4, 5, 8]) # 첫 번째 인자는 요청할 데이터의 종류를 말하는 5. 두 번째 인자는 어떤 인자를 요청할 지. 5는 종가. 
instStockChart.SetInputValue(6, ord('D')) # 첫 번째 인자는 차트의 종류를 말하는 6. 두 번째 인자는 어떤 차트인지. ord('D')는 일단위
instStockChart.SetInputValue(9, ord('1')) # 첫 번째 인자는 수정주가의 반영여부를 말하는 9, 두 번째 인자는 수정주가를 의미하는 ord('1')

In [24]:
instStockChart.BlockRequest()

0

In [25]:
print("""
Header information

Ticker: {ticker}
Field names: {field_names}
Data count: {data_count}
""".format(
    ticker=instStockChart.GetHeaderValue(0),
    field_names=instStockChart.GetHeaderValue(2),
    data_count=instStockChart.GetHeaderValue(3),
))



Header information

Ticker: A069500
Field names: ('날짜', '시가', '고가', '저가', '종가', '거래량')
Data count: 1000



In [26]:
data_count = instStockChart.GetHeaderValue(3)
fields_count = instStockChart.GetHeaderValue(1)
print(data_count, fields_count)

1000 6


In [27]:
kodex200 = []

for i in range(data_count):
    date = instStockChart.GetDataValue(0, i)
    open_price = instStockChart.GetDataValue(1, i)
    high_price = instStockChart.GetDataValue(2, i)
    low_price = instStockChart.GetDataValue(3, i)
    close_price = instStockChart.GetDataValue(4, i)
    volume = instStockChart.GetDataValue(5, i)
    
    row_dic = {}
    row_dic['date'] = date
    row_dic['open'] = open_price
    row_dic['high'] = high_price
    row_dic['low'] = low_price
    row_dic['close'] = close_price
    row_dic['volume'] = volume
    
    kodex200.append(row_dic)

In [28]:
# TODO: 변수 네이밍 바꿔주자 _df 붙일 대상 정하고 time series와 아닌것 구분

kodex200_ts_df = pd.DataFrame(kodex200)
kodex200_ts_df.head()

,date,open,high,low,close,volume
0,20200130,29695,29765,29375,29410,2718556
1,20200129,29764,29889,29585,29735,2968338
2,20200128,29849,30004,29485,29635,8162182
3,20200123,30768,30808,30538,30563,5616217
4,20200122,30498,30997,30483,30967,6071067


In [29]:
kodex200_ts_df.date = pd.to_datetime(kodex200_ts_df.date, format='%Y%m%d')
kodex200_ts_df = kodex200_ts_df.set_index('date')
kodex200_ts_df.head()

,open,high,low,close,volume
date,,,,,
2020-01-30,29695,29765,29375,29410,2718556
2020-01-29,29764,29889,29585,29735,2968338
2020-01-28,29849,30004,29485,29635,8162182
2020-01-23,30768,30808,30538,30563,5616217
2020-01-22,30498,30997,30483,30967,6071067


In [30]:
kodex200_ts_df.to_pickle('./data_files/kodex200_ts_df.pkl')

## Minute data 얻기. (tick은 너무 작으니까.)

In [37]:
# 종목과 조회조건 등을 설정. 

instStockChart.SetInputValue(0, code) # 첫 인자는 데이터 타입, 두 번째 인자는 입력 데이터 값. 0은 종목코드, A003540은 종목코드. 
instStockChart.SetInputValue(1, ord('2')) # 첫 번째 인자는 기간/요청 설정을 말하는 1. 두 번째 인자는 기간으로 요청 시 1, 개수로 요청시 2. 
instStockChart.SetInputValue(4, 1000) # 첫 번째 인자는 요청개수 설정을 말하는 4. 두 번째 인자는 실제로 몇 개 요청할 것인지 개수. (여기선 10개)
instStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 8]) # 첫 번째 인자는 요청할 데이터의 종류를 말하는 5. 두 번째 인자는 어떤 인자를 요청할 지. 5는 종가. 
instStockChart.SetInputValue(6, ord('m')) # 첫 번째 인자는 차트의 종류를 말하는 6. 두 번째 인자는 어떤 차트인지. ord('D')는 일단위
instStockChart.SetInputValue(9, ord('1')) # 첫 번째 인자는 수정주가의 반영여부를 말하는 9, 두 번째 인자는 수정주가를 의미하는 ord('1')

In [38]:
instStockChart.BlockRequest()

0

In [39]:
print("""
Header information

Ticker: {ticker}
Field names: {field_names}
Data count: {data_count}
""".format(
    ticker=instStockChart.GetHeaderValue(0),
    field_names=instStockChart.GetHeaderValue(2),
    data_count=instStockChart.GetHeaderValue(3),
))



Header information

Ticker: A069500
Field names: ('날짜', '시간', '시가', '고가', '저가', '종가', '거래량')
Data count: 1000



In [40]:
data_count = instStockChart.GetHeaderValue(3)
fields_count = instStockChart.GetHeaderValue(1)
print(data_count, fields_count)

1000 7


In [43]:
kodex200 = []

for i in range(data_count):
    date = instStockChart.GetDataValue(0, i)
    time = instStockChart.GetDataValue(1, i)
    open_price = instStockChart.GetDataValue(2, i)
    high_price = instStockChart.GetDataValue(3, i)
    low_price = instStockChart.GetDataValue(4, i)
    close_price = instStockChart.GetDataValue(5, i)
    volume = instStockChart.GetDataValue(6, i)
    
    row_dic = {}
    row_dic['date'] = date
    row_dic['time'] = time
    row_dic['open'] = open_price
    row_dic['high'] = high_price
    row_dic['low'] = low_price
    row_dic['close'] = close_price
    row_dic['volume'] = volume
    
    kodex200.append(row_dic)

In [44]:
# TODO: 변수 네이밍 바꿔주자 _df 붙일 대상 정하고 time series와 아닌것 구분

kodex200_ts_df = pd.DataFrame(kodex200)
kodex200_ts_df.head()

,date,time,open,high,low,close,volume
0,20200130,1351,29295,29295,29285,29285,190
1,20200130,1350,29285,29295,29275,29290,4316
2,20200130,1349,29270,29290,29270,29280,6322
3,20200130,1348,29260,29275,29255,29270,7503
4,20200130,1347,29265,29265,29250,29260,9275


In [45]:
kodex200_ts_df.tail()

,date,time,open,high,low,close,volume
995,20200128,958,29764,29764,29749,29754,5198
996,20200128,957,29754,29759,29749,29759,29256
997,20200128,956,29764,29769,29749,29749,22529
998,20200128,955,29784,29789,29764,29764,30621
999,20200128,954,29794,29794,29784,29789,14406
